### Charger les données

Avant de nettoyer les données, il faut les charger :

In [ ]:
import pandas as pd

# Charger un fichier CSV
df = pd.read_csv("data.csv")
pd.set_option('display.max_columns', None) #permet d'afficher toutes les colonnes

# Afficher les premières lignes
print(df.head())

On peut aussi charger d'autres formats comme Excel ```(pd.read_excel())```, JSON ```(pd.read_json())``` ou SQL ```(pd.read_sql())```.

### Comprendre les données
Avant de nettoyer, il faut examiner l'état des données :

In [ ]:
# Afficher des infos générales
print(df.info())

# Résumé statistique des colonnes numériques
print(df.describe())

# Voir les valeurs uniques d'une colonne
print(df["colonne"].unique())


### Modifier les données

On peut réaliser plusieurs modifications pour plus de lisibilité.

In [ ]:
df.rename(columns={
            'price': 'prix(million)', 
            'area': 'air(m2)',
            'bedrooms' : 'chambres',
            'bathrooms' : 'sdb',
            'stories' : 'étages',
            'mainroad' : 'route principale',
            'guestroom' : 'chambre ami',
            'basement' : 'sous sol',
            'hotwaterheating' : 'chauffage au gaz',
            'airconditioning' : 'climatisation',
            'parking' : 'parking',
            'prefarea' : 'résidentiel',
            'furnishingstatus' : 'meublé',
            }, inplace=True)

### Gérer les valeurs manquantes
Les valeurs manquantes sont un problème courant. Voici comment les détecter et les traiter :  

- Détection des valeurs manquantes :

In [ ]:
print(df.isnull().sum())  # Compter le nombre de NaN par colonne

- Supprimer les lignes ou colonnes avec des valeurs manquantes :

In [ ]:
df = df.dropna()  # Supprime toutes les lignes avec au moins un NaN
df = df.dropna(axis=1)  # Supprime les colonnes avec au moins un NaN

- Remplacer les valeurs manquantes :

In [ ]:
df["colonne"] = df["colonne"].fillna("valeur_par_défaut")  # Remplir avec une valeur spécifique
df["colonne"] = df["colonne"].fillna(df["colonne"].mean())  # Remplir avec la moyenne

### Gérer les doublons
Les doublons peuvent fausser les analyses :

In [ ]:
# Détection des doublons
print(df.duplicated().sum())
# ou aussi
df.loc[df.duplicated()]

# Suppression des doublons
df = df.drop_duplicates()


### Correction des types de données
Parfois, les données sont mal typées :

In [ ]:
# Convertir une colonne en numérique
df["colonne"] = pd.to_numeric(df["colonne"], errors="coerce")

# Convertir une colonne en date
df["date_colonne"] = pd.to_datetime(df["date_colonne"], errors="coerce")


### Standardisation et Normalisation des Données
Les données peuvent être dans des unités différentes ou avoir des échelles variées. Il est souvent nécessaire de les standardiser.

- Standardisation (centrage-réduction, moyenne = 0, écart-type = 1) :

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df[["colonne1", "colonne2"]] = scaler.fit_transform(df[["colonne1", "colonne2"]])


- Normalisation (valeurs entre 0 et 1) :

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df[["colonne1", "colonne2"]] = scaler.fit_transform(df[["colonne1", "colonne2"]])


### Détection et Suppression des Valeurs Aberrantes (Outliers)
Les valeurs aberrantes peuvent fausser les analyses statistiques.

- Utilisation de l'IQR (Interquartile Rang, 50 % des valeurs centrales) :

In [ ]:
Q1 = df["colonne"].quantile(0.25)
Q3 = df["colonne"].quantile(0.75)
IQR = Q3 - Q1

df = df[~((df["colonne"] < (Q1 - 1.5 * IQR)) | (df["colonne"] > (Q3 + 1.5 * IQR)))]

- Utilisation de l'écart-type (variabilité des données) :

In [ ]:
mean = df["colonne"].mean()
std = df["colonne"].std()

df = df[(df["colonne"] > mean - 3 * std) & (df["colonne"] < mean + 3 * std)]

### Nettoyage des Données Textuelles
Les données textuelles nécessitent souvent un prétraitement, notamment pour du NLP ou des analyses.

- Suppression des espaces en trop :

In [ ]:
df["colonne"] = df["colonne"].str.strip()

- Mise en minuscule :

In [ ]:
df["colonne"] = df["colonne"].str.lower()

- Suppression des caractères spéciaux :

In [ ]:
import re
df["colonne"] = df["colonne"].apply(lambda x: re.sub(r"[^a-zA-Z0-9 ]", "", x))

- Suppression des stopwords (mots inutiles) avec nltk :

In [ ]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words("french"))

df["colonne"] = df["colonne"].apply(lambda x: " ".join([word for word in x.split() if word not in stop_words]))

### Détection et Correction des Fautes de Saisie
On peut utiliser ````fuzzywuzzy```` pour corriger des noms mal écrits.

In [ ]:
from fuzzywuzzy import process

choices = ["Paris", "Marseille", "Lyon"]
df["colonne_corrigée"] = df["colonne"].apply(lambda x: process.extractOne(x, choices)[0])

### Encodage des Variables Catégoriques
Si une colonne contient des catégories, il faut l'encoder en numérique :

- Encodage One-Hot (binaire) :

In [ ]:
df = pd.get_dummies(df, columns=["categorie"], drop_first=True)

- Encodage Ordinal :

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

encoder = OrdinalEncoder(categories=[["Bas", "Moyen", "Élevé"]])
df["colonne"] = encoder.fit_transform(df[["colonne"]])

### Fusion et Regroupement de Données
Parfois, il faut combiner plusieurs sources de données.

- Jointure entre deux DataFrames :

In [ ]:
df_final = df1.merge(df2, on="clé_commune", how="left")

- Concaténation verticale :

In [ ]:
df_final = pd.concat([df1, df2], axis=0)

- Agrégation des données :

In [ ]:
df_grouped = df.groupby("colonne_catégorique").agg({"colonne_numérique": "mean"})

### Automatisation des Nettoyages avec des Pipelines
Pour structurer le processus, on peut utiliser ```sklearn.pipeline```.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

pipeline = Pipeline([
    ("imputation", SimpleImputer(strategy="mean")),
    ("normalisation", StandardScaler())
])

df[["colonne1", "colonne2"]] = pipeline.fit_transform(df[["colonne1", "colonne2"]])